# Hands On

As seen in the video, it's possible to use `git bisect` to find which commit introduced a bug. Let's do it by ourselves on a sample repository.

## Git

Firstly we need to install and setup git.

In [ ]:
%%bash
apt update
apt install git
git config --global user.name 'John Doe'
git config --global user.email johndoe@example.com

## Sample Repository

Here we will create a repository from scract and commit some changes. In one of the changes we will introduce a bug that later will be identified by using `git bisect`.

The repository will contain a dictionary of words stored in a file called `dict.txt`. Each commit will add or replace a word in the dictionary and by the end we expect a set of words to be present in the file.

In [ ]:
!mkdir repo
%cd repo
!git init

### Initial Commit

In [ ]:
%%bash
echo 'phone' >> dict.txt
git add -A
git commit -m 'add phone'

###Second Commit

In [ ]:
%%bash
echo 'call' >> dict.txt
git add -A
git commit -sm 'add call'

### Third Commit

In [ ]:
%%bash
echo 'people' >> dict.txt
git add -A
git commit -sm 'add people'

### Fourth Commit

In [ ]:
%%bash
echo 'all' >> dict.txt
git add -A
git commit -sm 'add all'

### Fifth Commit

In [ ]:
%%bash
echo 'meeting' >> dict.txt
git add -A
git commit -sm 'add meeting'

### Sixth Commit

In [ ]:
%%bash
sed -E 's/all/every/g' -i dict.txt
git add -A
git commit -sm 'replace all with every'

### Seventh Commit

In [ ]:
%%bash
sed -E 's/phone/cellphone/g' -i dict.txt
git add -A
git commit -sm 'replace phone with cellphone'

## Testing

Now that we have commited all the changes, we expect some words to be in the dictionary. Let's check if they are!

In [ ]:
%%bash
for WORD in call cellphone every meeting people
do
  grep -q $WORD ./dict.txt && printf 'PASS' || printf 'FAIL'
  echo " $WORD"
done

Unfortunately we the word `call` was expected to be in the dictionary but it isn't.

## Debuging

Now we can use `git bisect` to check when that happened. But first, let's check our commit history to see when we added `call` to the dictionary.

In [ ]:
!git log --oneline

It seems we added the word 5 commits ago, so for sure the world is in the dictionary at that commit. Now we can start the bisect!

In [ ]:
%%bash
git bisect start
git bisect bad HEAD    # current commit is a bad commit
git bisect good HEAD~5 # 5 commits ago is a good commit

Now we are in a different commit and we need to check if the word `call` is in the dictionary or not.

In [ ]:
!grep -q 'call' ./dict.txt && printf 'PASS' || printf 'FAIL'

As we can see, the word is there. So we mark this commit as good.

In [ ]:
!git bisect good

Git has moved us to another commit and we need to check again for the existence of the word.

In [ ]:
!grep -q 'call' ./dict.txt && printf 'PASS' || printf 'FAIL'

Now the check as failed. We need to mark this commit as a bad one.

In [ ]:
!git bisect bad

We run the test again.

In [ ]:
!grep -q 'call' ./dict.txt && printf 'PASS' || printf 'FAIL'

This time the commit is a good one.

In [ ]:
!git bisect good

After classifing some commits for git, it has found which one introduced the problem. And as we can see, it was the one that we replaced the world `all` with `every`.

Looking back at the commands we've executed on that commit, we can se that the regular expression that we used for the replacement was too broad and `call` was mistakenly rewritten as `cevery`.

We have successfully used `git bisect` to identify a commit that introducted a bug on our repository! Now let's run a final command to reset our repository to its original state.

In [ ]:
!git bisect reset